In [27]:
import os
import re
import gzip
import numpy as np
import pandas as pd

In [24]:
from collections import Counter
from tqdm.auto import tqdm

# Enable progress_apply for pandas
tqdm.pandas()

In [3]:
tsv_path = '/home/chb3333/yulab/chb3333/gem-patho/data_extraction/deduplicated_wxs_sample_sheet_clean.tsv'
wxs_tsv = pd.read_csv(tsv_path, sep='\t')

In [4]:
wxs_tsv

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type
0,076ef907-8f67-4822-9c76-3a902dce9a6d,46371375-b578-4ca6-ac74-5f534662f8d3.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TARGET-AML,TARGET-20-PATABB,"TARGET-20-PATABB-14A, TARGET-20-PATABB-04A","Bone Marrow Normal, Recurrent Blood Derived Ca..."
1,8cd36e93-09c6-4a17-a417-7139a56dbf7f,ff03b454-36d7-4d83-85c1-d5d3280f1305.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TARGET-AML,TARGET-20-PANVGP,"TARGET-20-PANVGP-09A, TARGET-20-PANVGP-14A","Primary Blood Derived Cancer - Bone Marrow, Bo..."
2,b746f7e7-9d92-49db-b2a8-d753f8e65709,e196b6da-38e0-4f3a-8dd6-27b4cda36ca4.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TARGET-AML,TARGET-20-PANVGP,"TARGET-20-PANVGP-14A, TARGET-20-PANVGP-04A","Bone Marrow Normal, Recurrent Blood Derived Ca..."
3,7df0dae5-9ffc-423f-b7f8-d45bf67d5f51,9a0eb32a-f2c3-46a7-926c-448a6026c486.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TARGET-AML,TARGET-20-PANLIZ,"TARGET-20-PANLIZ-04A, TARGET-20-PANLIZ-14A","Recurrent Blood Derived Cancer - Bone Marrow, ..."
4,b429fcc1-2b59-4b4c-a472-fb27758f6249,261a3bda-f09e-46dc-8ecd-9091c6775e88.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TARGET-AML,TARGET-20-PANSBH,"TARGET-20-PANSBH-04A, TARGET-20-PANSBH-14A","Recurrent Blood Derived Cancer - Bone Marrow, ..."
...,...,...,...,...,...,...,...,...
17318,e37d27ab-bbe5-4d2a-b589-b1d8504ed5bb,3b273a49-d6cd-449f-a641-5e8247c32087.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TARGET-AML,TARGET-20-PATELT,"TARGET-20-PATELT-09A, TARGET-20-PATELT-10A","Primary Blood Derived Cancer - Bone Marrow, Bl..."
17319,49820bc1-7500-475d-8c67-6d8a25f825f6,e1efed43-be3f-41aa-a85b-a248504bbbd4.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TARGET-AML,TARGET-20-PASFEW,"TARGET-20-PASFEW-14A, TARGET-20-PASFEW-09A","Bone Marrow Normal, Primary Blood Derived Canc..."
17320,04bd9936-fd84-4314-8eda-086652176843,b371125c-80f3-4242-be0f-3b123048c780.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TARGET-AML,TARGET-20-PAMYAS,"TARGET-20-PAMYAS-14A, TARGET-20-PAMYAS-09A","Bone Marrow Normal, Primary Blood Derived Canc..."
17321,66d43982-5d14-4a4a-8bc1-e45b77a1dc9f,6021d1e1-af33-4fca-838b-cffe5a8ac2cd.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TARGET-AML,TARGET-20-PANGTF,"TARGET-20-PANGTF-14A, TARGET-20-PANGTF-04A","Bone Marrow Normal, Recurrent Blood Derived Ca..."


In [5]:
csv_path = '/home/chb3333/yulab/chb3333/gem-patho/data_extraction/TCGA-CDR-SupplementalTableS1.csv'
wxs_csv = pd.read_csv(csv_path)
cols_to_extract = ['bcr_patient_barcode', 'type', 'vital_status', 'OS', 'OS.time']
wxs_csv_subset = wxs_csv[cols_to_extract]


In [6]:
merged_df = wxs_tsv.merge(wxs_csv_subset, left_on='Case ID', right_on='bcr_patient_barcode', how='left')
merged_df.drop(columns=['bcr_patient_barcode'], inplace=True)

In [7]:
merged_tsv = merged_df[merged_df['Project ID'].str.contains('TCGA', regex=False, na=False)]

In [8]:
merged_tsv

,File ID,File Name,Data Category,Data Type,Project ID,Case ID,Sample ID,Sample Type,type,vital_status,OS,OS.time
9,55d8f676-56d5-4740-bf7e-44eb38fcae96,cfbdf179-9e1a-4356-add3-9b3b7b040abd.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-OV,TCGA-42-2588,"TCGA-42-2588-10A, TCGA-42-2588-01A","Blood Derived Normal, Primary Tumor",OV,Dead,1.0,880.0
10,05129459-a31a-4d01-bcea-5ff828f150a4,ba454378-50aa-4a12-b76a-0f23b7a79ced.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-OV,TCGA-20-1686,"TCGA-20-1686-01A, TCGA-20-1686-10A","Primary Tumor, Blood Derived Normal",OV,Alive,0.0,89.0
11,5788e994-a06b-43ce-be6a-018af892fe13,3857627a-af9a-4c8e-b00c-4182ec8ae451.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-OV,TCGA-13-0916,"TCGA-13-0916-10A, TCGA-13-0916-01A","Blood Derived Normal, Primary Tumor",OV,Alive,0.0,1785.0
12,db3393db-a672-474c-b460-9321d06d7b34,b0ec2aa4-6940-49bc-b8a6-ee2b02baeb80.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-OV,TCGA-61-1740,"TCGA-61-1740-01A, TCGA-61-1740-11A","Primary Tumor, Solid Tissue Normal",OV,Dead,1.0,74.0
13,4778df07-cc7f-428a-8f7d-a0c20b07f276,bc865de1-66b9-4766-8c03-f3b107f1c9aa.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-ESCA,TCGA-R6-A8WG,"TCGA-R6-A8WG-10A, TCGA-R6-A8WG-01A","Blood Derived Normal, Primary Tumor",ESCA,Dead,1.0,386.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17307,de2ad78e-ed48-4de9-bdac-02eaf10c0d6a,8c0c889a-04b1-4802-9160-4e7d454ced4e.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-UCEC,TCGA-BS-A0T9,"TCGA-BS-A0T9-01A, TCGA-BS-A0T9-10C","Primary Tumor, Blood Derived Normal",UCEC,Dead,1.0,1428.0
17308,59f30b08-f0b2-490f-900a-ae683c9b7a40,efecc9ec-e20e-4acf-a387-144c03578371.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-UCEC,TCGA-AJ-A3BK,"TCGA-AJ-A3BK-10A, TCGA-AJ-A3BK-01A","Blood Derived Normal, Primary Tumor",UCEC,Alive,0.0,1031.0
17309,6eccff54-9e21-4a70-846f-fd538fc70eb1,453f7c70-e68f-4cc4-b9e4-a8929fb02b91.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-UCEC,TCGA-EY-A547,"TCGA-EY-A547-01A, TCGA-EY-A547-10A","Primary Tumor, Blood Derived Normal",UCEC,Alive,0.0,545.0
17310,7a0890e7-f868-4182-8908-f531a4baac16,1189e3e2-1944-4f26-b711-5f4ff707c874.wxs.aliqu...,Simple Nucleotide Variation,Masked Somatic Mutation,TCGA-UCEC,TCGA-AX-A3FT,"TCGA-AX-A3FT-01A, TCGA-AX-A3FT-10A","Primary Tumor, Blood Derived Normal",UCEC,Alive,0.0,6.0


In [9]:
print(f"Total unique extracted Case IDs: {len(merged_tsv['Case ID'].unique())}")


Total unique extracted Case IDs: 10190


In [11]:
gene_list_path = "/home/chb3333/yulab/chb3333/gem-patho/data_extraction/cancer_gene_list_selection/combined_genelist.csv"
cancer_genes_df = pd.read_csv(gene_list_path)
selected_genes = list(cancer_genes_df["Gene Symbol"])

In [15]:
# Direct extraction of mutational information

base_folder = "/home/chb3333/scratch/GDC-WGS-WXS/wxs"

In [28]:

# =============================================================================
# Precompute the average missense PolyPhen score across all mutation files.
# =============================================================================
def compute_average_missense_score(merged_df, base_folder):
    missense_scores = []
    for idx, row in tqdm(merged_df.iterrows(), total=len(merged_df), desc="Computing average missense score"):
        file_id = row["File ID"]
        file_name = row["File Name"]
        file_path = os.path.join(base_folder, file_id, file_name)
        try:
            with gzip.open(file_path, 'rt') as f:
                header = None
                for line in f:
                    if line.startswith("#"):
                        continue
                    else:
                        header = line.strip().split("\t")
                        break
                if header is None:
                    continue
                try:
                    var_class_idx = header.index("Variant_Classification")
                    polyphen_idx = header.index("PolyPhen")
                except ValueError:
                    continue
                for line in f:
                    if not line.strip():
                        continue
                    fields = line.strip().split("\t")
                    mutation_type = fields[var_class_idx]
                    if mutation_type == "Missense_Mutation":
                        raw_pp = fields[polyphen_idx]
                        if raw_pp:
                            match = re.search(r'\(([\d\.]+)\)', raw_pp)
                            if match:
                                try:
                                    score = float(match.group(1))
                                    if score > 0:
                                        missense_scores.append(score)
                                except ValueError:
                                    continue
        except Exception as e:
            print(f"Error processing file {file_path} for average score computation: {e}")
    if missense_scores:
        return np.mean(missense_scores)
    else:
        return 1.0

average_missense_score = compute_average_missense_score(merged_tsv, base_folder)
print("Average missense PolyPhen score (for nonzero scores):", average_missense_score)


Computing average missense score: 100%|██████████| 10190/10190 [01:01<00:00, 164.40it/s]


Average missense PolyPhen score (for nonzero scores): 0.5600329566861582


In [29]:

# =============================================================================
# Define the mapping function to obtain a PolyPhen score from a mutation.
# =============================================================================
def map_polyphen(mutation_type, polyphen_str, avg_score):
    """
    Maps the PolyPhen score for a given mutation based on its type.
    
    Parameters:
      mutation_type (str): The type of mutation (e.g., "Missense_Mutation", "Nonsense_Mutation", etc.)
      polyphen_str (str or None): A string like "benign(0.007)" or "probably_damaging(0.998)" or None.
      avg_score (float): The average missense score computed from valid missense mutations.
    
    Returns:
      float: The mapped PolyPhen score.
    """
    if mutation_type in ["Silent", "5'UTR", "3'UTR", "5'Flank", "3'Flank", "RNA"]:
        return 0.0
    elif mutation_type == "Intron":
        return avg_score
    elif mutation_type == "Missense_Mutation":
        if polyphen_str:
            match = re.search(r'\(([\d\.]+)\)', polyphen_str)
            if match:
                return float(match.group(1))
        return avg_score
    else:
        return 1.0

# =============================================================================
# Process a single mutation file and extract information for selected genes.
# =============================================================================
def process_mutation_file(file_path, selected_genes, avg_score):
    """
    Process a gzipped mutation file.
    Returns three dictionaries:
      - gene_mutated: mapping each selected gene to 0 (not mutated) or 1 (mutated)
      - gene_polyphen: mapping each selected gene to the aggregated PolyPhen score (maximum across mutations)
      - missing_counter: a Counter with counts of missing PolyPhen scores by mutation type (for this file)
    """
    gene_mutated = {gene: 0 for gene in selected_genes}
    # We will aggregate individual mapped scores in a list for each gene.
    gene_scores = {gene: [] for gene in selected_genes}
    missing_counter = Counter()
    
    try:
        with gzip.open(file_path, 'rt') as f:
            header = None
            for line in f:
                if line.startswith("#"):
                    continue
                else:
                    header = line.strip().split("\t")
                    break
            if header is None:
                return gene_mutated, {gene: 0.0 for gene in selected_genes}, missing_counter
            try:
                hugo_idx = header.index("Hugo_Symbol")
                polyphen_idx = header.index("PolyPhen")
                var_class_idx = header.index("Variant_Classification")
            except ValueError as e:
                print(f"Expected column not found in file {file_path}: {e}")
                return gene_mutated, {gene: 0.0 for gene in selected_genes}, missing_counter
            
            for line in f:
                if not line.strip():
                    continue
                fields = line.strip().split("\t")
                gene = fields[hugo_idx]
                if gene in selected_genes:
                    gene_mutated[gene] = 1  # mark as mutated
                    mutation_type = fields[var_class_idx]
                    raw_pp = fields[polyphen_idx]
                    score = map_polyphen(mutation_type, raw_pp, avg_score)
                    # Check whether the raw_pp provided a valid number.
                    valid = False
                    if raw_pp:
                        m = re.search(r'\(([\d\.]+)\)', raw_pp)
                        if m:
                            valid = True
                    if not valid:
                        missing_counter[mutation_type] += 1
                    gene_scores[gene].append(score)
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
    
    # For each gene, use the maximum score among its mutations (or 0.0 if none).
    gene_polyphen = {}
    for gene in selected_genes:
        if gene_scores[gene]:
            gene_polyphen[gene] = max(gene_scores[gene])
        else:
            gene_polyphen[gene] = 0.0
    
    return gene_mutated, gene_polyphen, missing_counter

# =============================================================================
# Process each row of merged_tsv and return the gene vectors and missing info.
# =============================================================================
def process_merged_row(row, base_folder, selected_genes, avg_score):
    """
    Constructs the file path using File ID and File Name, processes the mutation file,
    and returns:
      - mutated_genes: one-hot encoding (list) for each gene in selected_genes
      - polyphen_score: list of PolyPhen scores (aggregated by max) for each gene
      - missing_polyphen_types: dictionary of missing PolyPhen counts (aggregated across genes)
    """
    file_id = row["File ID"]
    file_name = row["File Name"]
    file_path = os.path.join(base_folder, file_id, file_name)
    
    gene_mutated, gene_polyphen, missing_counter = process_mutation_file(file_path, selected_genes, avg_score)
    mutated_list = [gene_mutated[gene] for gene in selected_genes]
    polyphen_list = [gene_polyphen[gene] for gene in selected_genes]
    
    # Convert the missing_counter (Counter) to a dict.
    missing_dict = dict(missing_counter)
    
    return pd.Series({
        "mutated_genes": mutated_list, 
        "polyphen_score": polyphen_list,
        "missing_polyphen_types": missing_dict
    })

In [30]:
merged_tsv[["mutated_genes", "polyphen_score", "missing_polyphen_types"]] = merged_tsv.progress_apply(
    lambda row: process_merged_row(row, base_folder, selected_genes, average_missense_score), axis=1
)


  0%|          | 10/10190 [00:00<01:44, 97.25it/s]

100%|██████████| 10190/10190 [01:51<00:00, 91.04it/s]
/tmp/ipykernel_213246/235295965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_tsv[["mutated_genes", "polyphen_score", "missing_polyphen_types"]] = merged_tsv.progress_apply(
/tmp/ipykernel_213246/235295965.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_tsv[["mutated_genes", "polyphen_score", "missing_polyphen_types"]] = merged_tsv.progress_apply(


In [ ]:
merged_tsv

In [35]:
save_path = '/home/chb3333/yulab/chb3333/gem-patho/data_extraction/patient_mutation_polyphen_vector_OStime/master_df.parquet'

merged_tsv.to_parquet(save_path)

In [32]:
unique_missing_types = set()

# Loop through each dictionary in the missing_polyphen_types column.
for missing_dict in merged_tsv["missing_polyphen_types"]:
    unique_missing_types.update(missing_dict.keys())

print("Unique missing PolyPhen mutation types:")
print(sorted(unique_missing_types))

Unique missing PolyPhen mutation types:
["3'Flank", "3'UTR", "5'Flank", "5'UTR", 'Frame_Shift_Del', 'Frame_Shift_Ins', 'In_Frame_Del', 'In_Frame_Ins', 'Intron', 'Missense_Mutation', 'Nonsense_Mutation', 'Nonstop_Mutation', 'RNA', 'Silent', 'Splice_Region', 'Splice_Site', 'Translation_Start_Site']


In [ ]:
# if mutation_type in ["Silent", "5'UTR", "3'UTR", "5'Flank", "3'Flank", "RNA"]:
#     return 0.0
# elif mutation_type == "Intron":
#     return avg_score
# elif mutation_type == "Missense_Mutation":
#     if polyphen_str:
#         match = re.search(r'\(([\d\.]+)\)', polyphen_str)
#         if match:
#             return float(match.group(1))
#     return avg_score
# else:
#     return 1.0